In [17]:
# import pandas (library for dataframes)
import pandas as pd
pd.set_option('display.max_columns', None)
# import listdir (library to list directories)
from os import listdir
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [18]:
from datetime import datetime, timedelta
import xlsxwriter
import inflect
import math

In [19]:
def breathing_calculations(file_name, file_suffix, file_location):

    # Check if file exists in file location
    file_exists = False
    for dir_file in listdir(file_location):
        if dir_file == file_name + file_suffix:
            file_exists = True
    
    if not file_exists:
        print("File: " + file_name + file_suffix + " does not exist in " + file_location)
        raise Exception("File: " + file_name + file_suffix + " does not exist in " + file_location)

    # Read in data from file into dataframe
    data = None
    column_names = ["row_number","cpu_date","cpu_time","site_time","period_time","Action","title",
                "description","Blank", "Breath_Num","first_beat_id","last_beat_id", "Ti","Te",
                "PIF","PEF", "TV", "EV","RT","MV", "f", "EIP", "EEP", "Penh", "EF50", "RH", 
                "Tbox", "Tbody", "Patm", "VCF", "AV", "Sr", "n"]
    with open(file_location + file_name + file_suffix, encoding='unicode_escape') as f:
        data = pd.read_csv(f, header=None, names=column_names, skip_blank_lines=True, na_filter=True, dtype='string', skipinitialspace=True, sep='\t')

    # row numbers when Action column is period-start
    period_start_rows = data[data["Action"] == "period-start"].index

    # start the data at the row of the first period-start Action
    first_period_start = int(period_start_rows[0])
    #print(first_period_start)

    # row numbers when Action column is period-stop
    period_stop_rows = data[data["Action"] == "period-stop "].index

    # end the data at the row of the last period-stop Action
    last_period_stop = int(period_stop_rows[-1])
    #print(last_period_stop)

    # change data to be from first period-start to last period-stop
    data = data.iloc[first_period_start:last_period_stop+1]

    # Drop unnecessary columns/rows
    data = data.drop(["first_beat_id","last_beat_id"], axis="columns")
    data = data.drop("Blank", axis="columns")
    data = data.drop(["row_number", "cpu_date", "cpu_time"], axis="columns")

    analyzer_row_numbers = data[data["Action"] == "analyzer-tuning"].index
    data = data.drop(analyzer_row_numbers)

    # drop 0 rows
    data = data[(data["Ti"] != "0") | (data["title"].notna())]

    # convert necessary columns to float types
    num_cols = len(data.columns)
    list_cols_to_convert_to_float = list(range(6, num_cols))
    for i in list_cols_to_convert_to_float:
        data.iloc[:, i] = data.iloc[:, i].map(float, na_action='ignore')

    # Reset index to start from 0
    data = data.reset_index(drop=True)

    # Period start and period stop rows change after reset_index, so reassign variables
    period_start_rows = data[data["Action"] == "period-start"].index
    period_stop_rows = data[data["Action"] == "period-stop "].index

    # Number of Breaths per condition
    num_breaths_per_condition = period_stop_rows - period_start_rows - 1
    #print(num_breaths_per_condition)

    # Get title condition for each period-start Action
    conditions = list(data[data["Action"] == "period-start"]["title"].values)
    #print(conditions)

    # Replace Ramp if exists
    if "Hypoxia" in conditions:
        hypoxia_index = conditions.index("Hypoxia")
        # Change Ramp to Ramp Up After Hypoxia (if exists)
        if conditions[hypoxia_index + 1] == "Ramp":
            conditions[hypoxia_index + 1] = "Ramp Up"

        # Change Ramp to Ramp Down Before Hypoxia (if exists)
        if conditions[hypoxia_index - 1] == "Ramp":
            conditions[hypoxia_index - 1] = "Ramp Down"

    # Keep all conditions not equal (ne) to Ramp
    # If ramp is removed, be sure to remove ramp data as well
    # conditions = list(filter("Ramp".__ne__, conditions))
    # print(conditions)

    # Loop through number of conditions to average data for each condition
    averaged_data = pd.DataFrame()
    for i in range(len(conditions)):
        condition_start_row = period_start_rows[i] + 1
        condition_stop_row = period_stop_rows[i]
        condition_rows_data = data.iloc[condition_start_row:condition_stop_row, 6:]
        filtered_TV = condition_rows_data[condition_rows_data["TV"] < 0.8]
        one_averaged_data = filtered_TV.mean()
        averaged_data = averaged_data.append(one_averaged_data, ignore_index=True)

    # Join Breaths Dataframe with Averages Dataframe
    breaths_dataframe = pd.DataFrame(num_breaths_per_condition, columns=["Breath_Count"])
    averaged_data = breaths_dataframe.join(averaged_data)

    # Differentiate Conditions using Dictionary to keep track of number of duplicates
    numbered_conditions = []
    number_dict = {}
    for condition in conditions:
        if condition in number_dict:
            number_dict[condition] += 1
            numbered_conditions.append(condition + " " + str(number_dict[condition]))
        else:
            number_dict[condition] = 1
            numbered_conditions.append(condition)

    # Join Conditions Dataframe with Averages Dataframe
    conditions_dataframe = pd.DataFrame(numbered_conditions, columns=["Conditions"])
    averaged_data = conditions_dataframe.join(averaged_data)

    # Duplicate file_directory row information averaged_data index # of times to join together
    file_dir_row = file_dir_data[file_dir_data["FileName"] == file_name]
    file_dir_row = file_dir_row.append([file_dir_row]*(len(averaged_data.index) - 1),ignore_index=True)

    # Join file_directory mouse info with averaged_data
    mouse_averaged_data = file_dir_row.join(averaged_data)
    return mouse_averaged_data

In [20]:
def condition_mean(cond1, cond2, row_name, averaged_data):
    if cond1 in averaged_data["Conditions"].values and cond2 in averaged_data["Conditions"].values:
        cond1_data = averaged_data[averaged_data["Conditions"] == cond1].iloc[:, 1:].values[0]
        cond2_data = averaged_data[averaged_data["Conditions"] == cond2].iloc[:, 1:].values[0]
        mean = [(cond1_i + cond2_i) / 2 for cond1_i, cond2_i in zip(cond1_data, cond2_data)]
        mean.insert(0, row_name)
        averaged_data.loc[len(averaged_data.index)] = mean
        return averaged_data

In [21]:
def condition_reduction(cond1, cond2, row_name, averaged_data):
    if cond1 in averaged_data["Conditions"].values and cond2 in averaged_data["Conditions"].values:
        # Get values of condition 1 data and condition 2 data as lists
        cond1_data = averaged_data[averaged_data["Conditions"] == cond1].iloc[:, 1:].values[0]
        cond2_data = averaged_data[averaged_data["Conditions"] == cond2].iloc[:, 1:].values[0]
        reduction = list((np.subtract(cond2_data, cond1_data) / cond1_data) * 100)
        reduction.insert(0, row_name)
        averaged_data.loc[len(averaged_data.index)] = reduction
        return averaged_data

In [22]:
# data - cleaned data
# title - the title/condition 5 min bin
# averaged data - averaged analysis table
def create_5min_bins(data, title, frequency, averaged_data):
    title_index = data[data["title"] == title].index

    # Excludes the period-start, period-stop rows
    title_data_rows = data.iloc[title_index[0]+1:title_index[1], :]

    # Convert period time to allow math calculations
    title_period_time = title_data_rows["period_time"]
    lst_time = []
    for period_time in title_period_time:
        new_time = None
        try:
            new_time = datetime.strptime(period_time, '%M:%S.%f')
        except:
            new_time = datetime.strptime(period_time, '%H:%M:%S.%f')
        
        lst_time.append(new_time)

    title_data_rows.loc[:, "period_time"] = lst_time

    # Set index to allow groupby 5 min aggregations
    title_data_rows = title_data_rows.set_index("period_time")

    # Convert analysis columns to floats to allow calculations for averages
    title_cols = len(title_data_rows.columns)
    list_cols_to_convert_to_float = list(range(4, title_cols))
    for i in list_cols_to_convert_to_float:
        title_data_rows.iloc[:, i] = title_data_rows.iloc[:, i].map(float, na_action='ignore')

    # Groupby 5 minute intervals, count number of rows in each 5 minute interval to determine breath count
    title_breath_num_count = title_data_rows.groupby(pd.Grouper(freq=str(frequency) + 'T')).agg({"Breath_Num":  "count"})
    title_breath_num_count = title_breath_num_count.reset_index(drop=True)

    # Take mean of analysis columns by 5 minute intervals
    title_analysis_5m_means = title_data_rows.iloc[:, 5:].resample(str(frequency) + "T").mean()
    title_analysis_5m_means = title_analysis_5m_means.reset_index(drop=True)

    # Create condition names for each 5 min interval for title (e.g. PI_first5, PI_second5, etc.)
    title_name = None
    if title == "Post Injection":
        title_name = "PI"
    elif title == "Recovery":
        title_name = "REC"
    else:
        title_name = "Title"

    num_title_rows = len(title_analysis_5m_means.index)
    list_5min_title_condition = []
    inflect_engine = inflect.engine()
    for i in range(1, num_title_rows + 1):
        list_5min_title_condition.append(title_name + "_" + inflect_engine.number_to_words(inflect_engine.ordinal(i)) + str(frequency))
    
    title_condition_df = pd.DataFrame(list_5min_title_condition, columns=["Conditions"])
    
    # Combine condition with breath count and analysis averages dataframe
    condition_breath = title_condition_df.join(title_breath_num_count)
    condition_breath_analysis = condition_breath.join(title_analysis_5m_means)
    condition_breath_analysis = condition_breath_analysis.rename(columns={"Breath_Num": "Breath_Count"})

    # Combine new 5Min Bin Dataframe with other averaged data conditions
    averages_and_5MinAnalysis = averaged_data.append(condition_breath_analysis)
    return averages_and_5MinAnalysis

In [23]:
# data - cleaned data
# title - the title/condition 5 min bin
# averaged data - averaged analysis table
def create_own_bins(data, title, start, end, averaged_data):
    title_index = data[data["title"] == title].index

    # Excludes the period-start, period-stop rows
    title_data_rows = data.iloc[title_index[0]+1:title_index[1], :]

    # Convert period time to allow math calculations
    title_period_time = title_data_rows["period_time"]
    lst_time = []
    for period_time in title_period_time:
        new_time = None
        try:
            new_time = datetime.strptime(period_time, '%M:%S.%f')
        except:
            new_time = datetime.strptime(period_time, '%H:%M:%S.%f')
        
        lst_time.append(new_time)

    title_data_rows.loc[:, "period_time"] = lst_time

    rows_between_time = title_data_rows[(title_data_rows["period_time"].dt.minute > start-1) & (title_data_rows["period_time"].dt.minute < end)]

    num_breaths = [int(list(rows_between_time["Breath_Num"])[-1]) - int(list(rows_between_time["Breath_Num"])[0])]
    num_breaths_df = pd.DataFrame(num_breaths, columns=["Breath_Num"])

    # Convert analysis columns to floats to allow calculations for averages
    title_cols = len(rows_between_time.columns)
    list_cols_to_convert_to_float = list(range(6, title_cols))
    for i in list_cols_to_convert_to_float:
        rows_between_time.iloc[:, i] = rows_between_time.iloc[:, i].map(float, na_action='ignore')

    mean_analysis_df = rows_between_time.iloc[:, 6:].mean(axis=0)
    analysis_average_df = pd.DataFrame(mean_analysis_df.to_dict(), index=[0])

    # Create condition names for each 5 min interval for title (e.g. PI_first5, PI_second5, etc.)
    title_name = None
    if title == "Post Injection":
        title_name = "PI"
    elif title == "Recovery":
        title_name = "REC"
    else:
        title_name = "Title"
    
    lst_title_name = [title_name + "_" + str(start) + "-" + str(end)]

    title_condition_df = pd.DataFrame(lst_title_name, columns=["Conditions"])
    
    # Combine condition with breath count and analysis averages dataframe
    condition_breath = title_condition_df.join(num_breaths_df)
    condition_breath_analysis = condition_breath.join(analysis_average_df)
    condition_breath_analysis = condition_breath_analysis.rename(columns={"Breath_Num": "Breath_Count"})

    # # Combine new 5Min Bin Dataframe with other averaged data conditions
    averages_and_5MinAnalysis = averaged_data.append(condition_breath_analysis)
    return averages_and_5MinAnalysis

In [24]:
# open file directory and save data to dataframe
with open('inputs/noninjection_marcus/___FILE DIRECTORY___.csv', encoding='unicode_escape') as f:
    file_dir_data = pd.read_csv(f, header=0, skip_blank_lines=True, na_filter=True, dtype='string', skipinitialspace=True, sep=',')
file_dir_data

,FileName,Chamber,Test_Number,Tester,Date_Tested,DOB,Mouse_ID,Sex,Coat_Color,Earclip,Mouse_Strain,Gene,Genotype,Gene2,Genotype2,Weight,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23
0,230131_chamber1_ajc7142_f22bl,1,1,MC,2023-01-31,<NA>,ajc7142,f,bl,22,filler,filler,filler,filler,filler,22.4,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,230131_chamber1_ajc7386_f03bl,1,1,MC,2023-01-31,<NA>,ajc7320,f,bl,10,filler,filler,filler,filler,filler,22,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,230131_chamber2_ajc7320_f10bl,2,1,MC,2023-01-31,<NA>,ajc7386,f,bl,3,filler,filler,filler,filler,filler,21.2,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,230131_chamber2_ajc7388_f30bl,2,1,MC,2023-01-31,<NA>,ajc7388,f,bl,30,filler,filler,filler,filler,filler,19,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,230207_chamber1_ajc7142_f22bl,1,2,MC + LC,2023-02-07,<NA>,ajc7142,f,bl,22,filler,filler,filler,filler,filler,22.6,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
5,230207_chamber2_ajc7320_f10bl,2,2,MC + LC,2023-02-07,<NA>,ajc7320,f,bl,10,filler,filler,filler,filler,filler,22.8,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
6,230207_chamber1_ajc7386_f03bl,1,2,MC + LC,2023-02-07,<NA>,ajc7386,f,bl,3,filler,filler,filler,filler,filler,20.7,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
7,230207_chamber2_ajc7388_f30bl,2,2,MC + LC,2023-02-07,<NA>,ajc7388,f,bl,30,filler,filler,filler,filler,filler,18.7,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
8,230221_chamber1_ajc6660_m03bl,1,1,TL,2023-02-21,<NA>,ajc6660,m,bl,3,filler,filler,filler,filler,filler,32.5,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
9,230221_chamber2_ajc6662_m22bl,2,1,TL,2023-02-21,<NA>,ajc6662,m,bl,22,filler,filler,filler,filler,filler,29,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [25]:
# Convert weight column to float for calculations
file_dir_data["Weight"] = file_dir_data["Weight"].map(float, na_action=None)

# Convert Date_Tested column to datetime for calculations
file_dir_data["Date_Tested"] = file_dir_data["Date_Tested"].map(lambda date: datetime.strptime(date, "%Y-%m-%d"))

# Convert DOB column to datetime for calculations
# file_dir_data["DOB"] = file_dir_data["DOB"].map(lambda date: datetime.datetime.strptime(date, "%Y-%m-%d"))

In [26]:
# Add Age column
# file_dir_data["Age"] = file_dir_data["Date_Tested"] - file_dir_data["DOB"]
#file_dir_data["Age"] = fillna()

In [27]:
analysis_names = ["Conditions","Breath_Count", "Ti","Te","PIF","PEF", 
                "TV", "EV","RT","MV", "f", "EIP", "EEP", "Penh", "EF50", 
                "RH", "Tbox","Tbody", "Patm","VCF", "AV", "Sr","n"]

master_data_columns = list(file_dir_data.columns.values) + analysis_names
master_data_columns

['FileName',
 'Chamber',
 'Test_Number',
 'Tester',
 'Date_Tested',
 'DOB',
 'Mouse_ID',
 'Sex',
 'Coat_Color',
 'Earclip',
 'Mouse_Strain',
 'Gene',
 'Genotype',
 'Gene2',
 'Genotype2',
 'Weight',
 'Unnamed: 16',
 'Unnamed: 17',
 'Unnamed: 18',
 'Unnamed: 19',
 'Unnamed: 20',
 'Unnamed: 21',
 'Unnamed: 22',
 'Unnamed: 23',
 'Conditions',
 'Breath_Count',
 'Ti',
 'Te',
 'PIF',
 'PEF',
 'TV',
 'EV',
 'RT',
 'MV',
 'f',
 'EIP',
 'EEP',
 'Penh',
 'EF50',
 'RH',
 'Tbox',
 'Tbody',
 'Patm',
 'VCF',
 'AV',
 'Sr',
 'n']

In [28]:
# Create empty master dataframe
master_data = pd.DataFrame(columns=master_data_columns)
master_data
# TODO: Should I do something with this?
# master_data$Date_Tested <- as.Date(master_data$Date_Tested,"%Y-%m-%d")
# master_data$DOB <- as.Date(master_data$DOB, "%m/%d/%y")

,FileName,Chamber,Test_Number,Tester,Date_Tested,DOB,Mouse_ID,Sex,Coat_Color,Earclip,Mouse_Strain,Gene,Genotype,Gene2,Genotype2,Weight,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Conditions,Breath_Count,Ti,Te,PIF,PEF,TV,EV,RT,MV,f,EIP,EEP,Penh,EF50,RH,Tbox,Tbody,Patm,VCF,AV,Sr,n


In [29]:
# Retrieve file list from file directory
file_list = file_dir_data["FileName"]
file_list

0       230131_chamber1_ajc7142_f22bl
1       230131_chamber1_ajc7386_f03bl
2       230131_chamber2_ajc7320_f10bl
3       230131_chamber2_ajc7388_f30bl
4       230207_chamber1_ajc7142_f22bl
5       230207_chamber2_ajc7320_f10bl
6       230207_chamber1_ajc7386_f03bl
7       230207_chamber2_ajc7388_f30bl
8       230221_chamber1_ajc6660_m03bl
9       230221_chamber2_ajc6662_m22bl
10      230221_chamber1_ajc7137_m10bl
11      230221_chamber2_ajc7138_m30bl
12      230228_chamber1_ajc6660_m03bl
13      230228_chamber2_ajc6662_m22bl
14      230228_chamber1_ajc7137_m10bl
15      230228_chamber2_ajc7138_m30bl
16      230426_chamber1_ajc7569_m22bl
17      230426_chamber2_ajc7572_m20bl
18      230426_chamber1_ajc7729_m01bl
19      230426_chamber2_ajc7730_m02bl
20      230502_chamber2_ajc7615_f20bl
21      230502_chamber2_ajc7620_f22bl
22      230602_chamber1_ajc7634_m02bl
23      230602_chamber2_ajc7635_m03bl
24      230602_chamber1_ajc7753_m20bl
25      230602_chamber2_ajc7754_m30bl
26      2306

In [30]:
# Apply breathing_calculations to each file
file_location = "inputs/noninjection_marcus/"
file_suffix = ".rf_1.iox.txt"
for file_name in file_list:
    print(file_name)
    master_data = master_data.append(breathing_calculations(file_name, file_suffix, file_location))
master_data

230131_chamber1_ajc7142_f22bl
230131_chamber1_ajc7386_f03bl
230131_chamber2_ajc7320_f10bl
230131_chamber2_ajc7388_f30bl
230207_chamber1_ajc7142_f22bl
230207_chamber2_ajc7320_f10bl
230207_chamber1_ajc7386_f03bl
230207_chamber2_ajc7388_f30bl
230221_chamber1_ajc6660_m03bl
230221_chamber2_ajc6662_m22bl
230221_chamber1_ajc7137_m10bl
230221_chamber2_ajc7138_m30bl
230228_chamber1_ajc6660_m03bl
230228_chamber2_ajc6662_m22bl
230228_chamber1_ajc7137_m10bl
230228_chamber2_ajc7138_m30bl
230426_chamber1_ajc7569_m22bl
230426_chamber2_ajc7572_m20bl
230426_chamber1_ajc7729_m01bl
230426_chamber2_ajc7730_m02bl
230502_chamber2_ajc7615_f20bl
230502_chamber2_ajc7620_f22bl
230602_chamber1_ajc7634_m02bl
230602_chamber2_ajc7635_m03bl
230602_chamber1_ajc7753_m20bl
230602_chamber2_ajc7754_m30bl
230602_chamber1_ajc7931_m03bl
230602_chamber2_ajc7934_m30bl
230609_chamber1_ajc7620_f22bl_1
230609_chamber2_ajc7569_m22bl_1
230609_chamber1_ajc7931_m03bl
230609_chamber2_ajc7753_m20bl
230610_chamber1_ajc7729_m01bl
230610

,FileName,Chamber,Test_Number,Tester,Date_Tested,DOB,Mouse_ID,Sex,Coat_Color,Earclip,Mouse_Strain,Gene,Genotype,Gene2,Genotype2,Weight,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Conditions,Breath_Count,Ti,Te,PIF,PEF,TV,EV,RT,MV,f,EIP,EEP,Penh,EF50,RH,Tbox,Tbody,Patm,VCF,AV,Sr,n
0,230131_chamber1_ajc7142_f22bl,1,1,MC,2023-01-31,<NA>,ajc7142,f,bl,22,filler,filler,filler,filler,filler,22.4,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Baseline,357,76.157303,209.938202,5.973006,3.016320,0.272949,0.269635,138.429775,60.716292,224.278090,1.839888,18.429775,0.312247,1.881966,-1.000000,-1.000000,-1.0,-1.000000,10.000000,50.662921,42.398876,1.0
1,230131_chamber1_ajc7142_f22bl,1,1,MC,2023-01-31,<NA>,ajc7142,f,bl,22,filler,filler,filler,filler,filler,22.4,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Ramp Down,94,68.372340,166.659574,6.432447,3.596596,0.301596,0.309362,130.574468,78.202128,258.478723,2.138298,10.648936,0.165426,2.559362,-1.000000,-1.000000,-1.0,-1.000000,10.000000,111.872340,80.882979,1.0
2,230131_chamber1_ajc7142_f22bl,1,1,MC,2023-01-31,<NA>,ajc7142,f,bl,22,filler,filler,filler,filler,filler,22.4,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Hypoxia,450,80.160000,196.151111,5.054267,2.926133,0.268778,0.269111,152.546667,59.144444,220.917778,1.162222,12.268889,0.179422,1.673400,-1.000000,-1.000000,-1.0,-1.000000,10.000000,187.031111,70.764444,1.0
3,230131_chamber1_ajc7142_f22bl,1,1,MC,2023-01-31,<NA>,ajc7142,f,bl,22,filler,filler,filler,filler,filler,22.4,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Ramp Up,8,77.875000,218.750000,5.802500,3.066250,0.308750,0.308750,171.500000,65.625000,213.625000,1.000000,12.625000,0.177500,1.767500,-1.000000,-1.000000,-1.0,-1.000000,10.000000,247.875000,76.500000,1.0
4,230131_chamber1_ajc7142_f22bl,1,1,MC,2023-01-31,<NA>,ajc7142,f,bl,22,filler,filler,filler,filler,filler,22.4,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Normoxia,1128,82.669627,216.980462,4.882806,2.428552,0.246226,0.244272,120.174067,50.935169,207.176732,1.192718,30.294849,0.433206,1.622735,-1.000000,-1.000000,-1.0,-1.000000,10.000000,393.095027,73.563055,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8,230623_chamber2_ajc7937_f31bl,2,1,DS,2023-06-23,<NA>,ajc7937,f,bl,31,Olfr78F,+/+,filler,filler,filler,21.7,<NA>,<NA>,<NA>,<NA>,No,<NA>,0,"Baseline, Hypoxia, Normoxia, Hypoxia, Normoxia...",Normoxia 2,915,89.170492,224.911475,3.368044,2.223967,0.170874,0.170831,127.032787,34.969399,204.224044,1.574863,25.626230,0.661148,1.400186,27.526448,20.158973,37.5,1009.000000,9.085027,85.309290,90.873224,1.0
9,230623_chamber2_ajc7937_f31bl,2,1,DS,2023-06-23,<NA>,ajc7937,f,bl,31,Olfr78F,+/+,filler,filler,filler,21.7,<NA>,<NA>,<NA>,<NA>,No,<NA>,0,"Baseline, Hypoxia, Normoxia, Hypoxia, Normoxia...",Ramp 3,249,88.261044,130.718876,3.550843,3.219197,0.206667,0.208394,66.594378,58.096386,281.385542,1.429719,20.939759,0.866827,2.314900,27.166667,20.160000,37.5,1009.000000,9.083333,25.036145,97.510040,1.0
10,230623_chamber2_ajc7937_f31bl,2,1,DS,2023-06-23,<NA>,ajc7937,f,bl,31,Olfr78F,+/+,filler,filler,filler,21.7,<NA>,<NA>,<NA>,<NA>,No,<NA>,0,"Baseline, Hypoxia, Normoxia, Hypoxia, Normoxia...",Hypercapnia,1374,86.153790,114.741254,4.240109,4.165539,0.250940,0.251800,62.762391,75.180029,299.561953,1.392857,15.008746,0.838448,2.858404,26.438848,20.184927,37.5,1009.000000,9.073280,170.078717,96.721574,1.0
11,230623_chamber2_ajc7937_f31bl,2,1,DS,2023-06-23,<NA>,ajc7937,f,bl,31,Olfr78F,+/+,filler,filler,filler,21.7,<NA>,<NA>,<NA>,<NA>,No,<NA>,0,"Baseline, Hypoxia, Normoxia, Hypoxia, Normoxia...",Ramp 4,220,83.621005,123.703196,4.302283,3.445114,0.219087,0.221233,67.223744,64.191781,291.223744,1.397260,16.972603,0.689224,2.370685,26.358904,20.291233,37.5,1009.077626,9.101370,26.538813,93.589041,1.0


In [31]:
# Normalize MV, TV by weight
master_data["MV/g"] = master_data["MV"] / master_data["Weight"]
master_data["TV/g"] = master_data["TV"] / master_data["Weight"]
master_data

,FileName,Chamber,Test_Number,Tester,Date_Tested,DOB,Mouse_ID,Sex,Coat_Color,Earclip,Mouse_Strain,Gene,Genotype,Gene2,Genotype2,Weight,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Conditions,Breath_Count,Ti,Te,PIF,PEF,TV,EV,RT,MV,f,EIP,EEP,Penh,EF50,RH,Tbox,Tbody,Patm,VCF,AV,Sr,n,MV/g,TV/g
0,230131_chamber1_ajc7142_f22bl,1,1,MC,2023-01-31,<NA>,ajc7142,f,bl,22,filler,filler,filler,filler,filler,22.4,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Baseline,357,76.157303,209.938202,5.973006,3.016320,0.272949,0.269635,138.429775,60.716292,224.278090,1.839888,18.429775,0.312247,1.881966,-1.000000,-1.000000,-1.0,-1.000000,10.000000,50.662921,42.398876,1.0,2.710549,0.012185
1,230131_chamber1_ajc7142_f22bl,1,1,MC,2023-01-31,<NA>,ajc7142,f,bl,22,filler,filler,filler,filler,filler,22.4,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Ramp Down,94,68.372340,166.659574,6.432447,3.596596,0.301596,0.309362,130.574468,78.202128,258.478723,2.138298,10.648936,0.165426,2.559362,-1.000000,-1.000000,-1.0,-1.000000,10.000000,111.872340,80.882979,1.0,3.491166,0.013464
2,230131_chamber1_ajc7142_f22bl,1,1,MC,2023-01-31,<NA>,ajc7142,f,bl,22,filler,filler,filler,filler,filler,22.4,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Hypoxia,450,80.160000,196.151111,5.054267,2.926133,0.268778,0.269111,152.546667,59.144444,220.917778,1.162222,12.268889,0.179422,1.673400,-1.000000,-1.000000,-1.0,-1.000000,10.000000,187.031111,70.764444,1.0,2.640377,0.011999
3,230131_chamber1_ajc7142_f22bl,1,1,MC,2023-01-31,<NA>,ajc7142,f,bl,22,filler,filler,filler,filler,filler,22.4,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Ramp Up,8,77.875000,218.750000,5.802500,3.066250,0.308750,0.308750,171.500000,65.625000,213.625000,1.000000,12.625000,0.177500,1.767500,-1.000000,-1.000000,-1.0,-1.000000,10.000000,247.875000,76.500000,1.0,2.929688,0.013783
4,230131_chamber1_ajc7142_f22bl,1,1,MC,2023-01-31,<NA>,ajc7142,f,bl,22,filler,filler,filler,filler,filler,22.4,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Normoxia,1128,82.669627,216.980462,4.882806,2.428552,0.246226,0.244272,120.174067,50.935169,207.176732,1.192718,30.294849,0.433206,1.622735,-1.000000,-1.000000,-1.0,-1.000000,10.000000,393.095027,73.563055,1.0,2.273891,0.010992
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8,230623_chamber2_ajc7937_f31bl,2,1,DS,2023-06-23,<NA>,ajc7937,f,bl,31,Olfr78F,+/+,filler,filler,filler,21.7,<NA>,<NA>,<NA>,<NA>,No,<NA>,0,"Baseline, Hypoxia, Normoxia, Hypoxia, Normoxia...",Normoxia 2,915,89.170492,224.911475,3.368044,2.223967,0.170874,0.170831,127.032787,34.969399,204.224044,1.574863,25.626230,0.661148,1.400186,27.526448,20.158973,37.5,1009.000000,9.085027,85.309290,90.873224,1.0,1.611493,0.007874
9,230623_chamber2_ajc7937_f31bl,2,1,DS,2023-06-23,<NA>,ajc7937,f,bl,31,Olfr78F,+/+,filler,filler,filler,21.7,<NA>,<NA>,<NA>,<NA>,No,<NA>,0,"Baseline, Hypoxia, Normoxia, Hypoxia, Normoxia...",Ramp 3,249,88.261044,130.718876,3.550843,3.219197,0.206667,0.208394,66.594378,58.096386,281.385542,1.429719,20.939759,0.866827,2.314900,27.166667,20.160000,37.5,1009.000000,9.083333,25.036145,97.510040,1.0,2.677253,0.009524
10,230623_chamber2_ajc7937_f31bl,2,1,DS,2023-06-23,<NA>,ajc7937,f,bl,31,Olfr78F,+/+,filler,filler,filler,21.7,<NA>,<NA>,<NA>,<NA>,No,<NA>,0,"Baseline, Hypoxia, Normoxia, Hypoxia, Normoxia...",Hypercapnia,1374,86.153790,114.741254,4.240109,4.165539,0.250940,0.251800,62.762391,75.180029,299.561953,1.392857,15.008746,0.838448,2.858404,26.438848,20.184927,37.5,1009.000000,9.073280,170.078717,96.721574,1.0,3.464517,0.011564
11,230623_chamber2_ajc7937_f31bl,2,1,DS,2023-06-23,<NA>,ajc7937,f,bl,31,Olfr78F,+/+,filler,filler,filler,21.7,<NA>,<NA>,<NA>,<NA>,No,<NA>,0,"Baseline, Hypoxia, Normoxia, Hypoxia, Normoxia...",Ramp 4,220,83.621005,123.703196,4.302283,3.445114,0.219087,0.221233,67.223744,64.191781,291.223744,1.397260,16.972603,0.68

In [32]:
# Prioritize MV, TV, f columns by rearranging column order
curr_column_order = master_data.columns.tolist()
breath_count_index = curr_column_order.index("Breath_Count")
curr_column_order.remove("MV")
curr_column_order.remove("TV")
curr_column_order.remove("f")
curr_column_order.remove("MV/g")
curr_column_order.remove("TV/g")
curr_column_order.insert(breath_count_index + 1, "MV")
curr_column_order.insert(breath_count_index + 2, "TV")
curr_column_order.insert(breath_count_index + 3, "f")
curr_column_order.insert(breath_count_index + 4, "MV/g")
curr_column_order.insert(breath_count_index + 5, "TV/g")
master_data = master_data[curr_column_order]
master_data

,FileName,Chamber,Test_Number,Tester,Date_Tested,DOB,Mouse_ID,Sex,Coat_Color,Earclip,Mouse_Strain,Gene,Genotype,Gene2,Genotype2,Weight,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Conditions,Breath_Count,MV,TV,f,MV/g,TV/g,Ti,Te,PIF,PEF,EV,RT,EIP,EEP,Penh,EF50,RH,Tbox,Tbody,Patm,VCF,AV,Sr,n
0,230131_chamber1_ajc7142_f22bl,1,1,MC,2023-01-31,<NA>,ajc7142,f,bl,22,filler,filler,filler,filler,filler,22.4,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Baseline,357,60.716292,0.272949,224.278090,2.710549,0.012185,76.157303,209.938202,5.973006,3.016320,0.269635,138.429775,1.839888,18.429775,0.312247,1.881966,-1.000000,-1.000000,-1.0,-1.000000,10.000000,50.662921,42.398876,1.0
1,230131_chamber1_ajc7142_f22bl,1,1,MC,2023-01-31,<NA>,ajc7142,f,bl,22,filler,filler,filler,filler,filler,22.4,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Ramp Down,94,78.202128,0.301596,258.478723,3.491166,0.013464,68.372340,166.659574,6.432447,3.596596,0.309362,130.574468,2.138298,10.648936,0.165426,2.559362,-1.000000,-1.000000,-1.0,-1.000000,10.000000,111.872340,80.882979,1.0
2,230131_chamber1_ajc7142_f22bl,1,1,MC,2023-01-31,<NA>,ajc7142,f,bl,22,filler,filler,filler,filler,filler,22.4,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Hypoxia,450,59.144444,0.268778,220.917778,2.640377,0.011999,80.160000,196.151111,5.054267,2.926133,0.269111,152.546667,1.162222,12.268889,0.179422,1.673400,-1.000000,-1.000000,-1.0,-1.000000,10.000000,187.031111,70.764444,1.0
3,230131_chamber1_ajc7142_f22bl,1,1,MC,2023-01-31,<NA>,ajc7142,f,bl,22,filler,filler,filler,filler,filler,22.4,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Ramp Up,8,65.625000,0.308750,213.625000,2.929688,0.013783,77.875000,218.750000,5.802500,3.066250,0.308750,171.500000,1.000000,12.625000,0.177500,1.767500,-1.000000,-1.000000,-1.0,-1.000000,10.000000,247.875000,76.500000,1.0
4,230131_chamber1_ajc7142_f22bl,1,1,MC,2023-01-31,<NA>,ajc7142,f,bl,22,filler,filler,filler,filler,filler,22.4,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Normoxia,1128,50.935169,0.246226,207.176732,2.273891,0.010992,82.669627,216.980462,4.882806,2.428552,0.244272,120.174067,1.192718,30.294849,0.433206,1.622735,-1.000000,-1.000000,-1.0,-1.000000,10.000000,393.095027,73.563055,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8,230623_chamber2_ajc7937_f31bl,2,1,DS,2023-06-23,<NA>,ajc7937,f,bl,31,Olfr78F,+/+,filler,filler,filler,21.7,<NA>,<NA>,<NA>,<NA>,No,<NA>,0,"Baseline, Hypoxia, Normoxia, Hypoxia, Normoxia...",Normoxia 2,915,34.969399,0.170874,204.224044,1.611493,0.007874,89.170492,224.911475,3.368044,2.223967,0.170831,127.032787,1.574863,25.626230,0.661148,1.400186,27.526448,20.158973,37.5,1009.000000,9.085027,85.309290,90.873224,1.0
9,230623_chamber2_ajc7937_f31bl,2,1,DS,2023-06-23,<NA>,ajc7937,f,bl,31,Olfr78F,+/+,filler,filler,filler,21.7,<NA>,<NA>,<NA>,<NA>,No,<NA>,0,"Baseline, Hypoxia, Normoxia, Hypoxia, Normoxia...",Ramp 3,249,58.096386,0.206667,281.385542,2.677253,0.009524,88.261044,130.718876,3.550843,3.219197,0.208394,66.594378,1.429719,20.939759,0.866827,2.314900,27.166667,20.160000,37.5,1009.000000,9.083333,25.036145,97.510040,1.0
10,230623_chamber2_ajc7937_f31bl,2,1,DS,2023-06-23,<NA>,ajc7937,f,bl,31,Olfr78F,+/+,filler,filler,filler,21.7,<NA>,<NA>,<NA>,<NA>,No,<NA>,0,"Baseline, Hypoxia, Normoxia, Hypoxia, Normoxia...",Hypercapnia,1374,75.180029,0.250940,299.561953,3.464517,0.011564,86.153790,114.741254,4.240109,4.165539,0.251800,62.762391,1.392857,15.008746,0.838448,2.858404,26.438848,20.184927,37.5,1009.000000,9.073280,170.078717,96.721574,1.0
11,230623_chamber2_ajc7937_f31bl,2,1,DS,2023-06-23,<NA>,ajc7937,f,bl,31,Olfr78F,+/+,filler,filler,filler,21.7,<NA>,<NA>,<NA>,<NA>,No,<NA>,0,"Baseline, Hypoxia, Normoxia, Hypoxia, Normoxia...",Ramp 4,220,64.191781,0.219087,291.223744,2.958147,0.010096,83.621005,123.703196,4.302283,3.445114,0.221233,67.223744,1.397

In [33]:
# Normalize MV and TV columns for Hypoxia/Normoxia
# TODO: WHAT TO DO WITH THIS???
# normalized = pd.DataFrame()
# normalized["MV/g"] = (master_data[master_data["Conditions"] == "Hypoxia"]["MV/g"] - master_data[master_data["Conditions"] == "Normoxia"]["MV/g"]) / master_data[master_data["Conditions"] == "Hypoxia"]["MV"]
# normalized["TV/g"] = (master_data[master_data["Conditions"] == "Hypoxia"]["TV/g"] - master_data[master_data["Conditions"] == "Normoxia"]["TV/g"]) / master_data[master_data["Conditions"] == "Hypoxia"]["TV"]
# normalized

In [34]:
# Make excel sheet easier to read by sorting by Mouse_ID and then Test_Number
master_data = master_data.sort_values(by=['Mouse_ID', 'Test_Number'])
master_data

,FileName,Chamber,Test_Number,Tester,Date_Tested,DOB,Mouse_ID,Sex,Coat_Color,Earclip,Mouse_Strain,Gene,Genotype,Gene2,Genotype2,Weight,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Conditions,Breath_Count,MV,TV,f,MV/g,TV/g,Ti,Te,PIF,PEF,EV,RT,EIP,EEP,Penh,EF50,RH,Tbox,Tbody,Patm,VCF,AV,Sr,n
0,230221_chamber1_ajc6660_m03bl,1,1,TL,2023-02-21,<NA>,ajc6660,m,bl,3,filler,filler,filler,filler,filler,32.5,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Baseline,900,65.576063,0.308859,211.802013,2.017725,0.009503,81.355705,221.458613,5.967136,3.317148,0.307114,162.508949,1.390380,16.211409,0.308960,2.102148,53.313647,20.231499,37.5,999.104027,9.538367,145.086130,66.394855,1.0
1,230221_chamber1_ajc6660_m03bl,1,1,TL,2023-02-21,<NA>,ajc6660,m,bl,3,filler,filler,filler,filler,filler,32.5,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Ramp Down,140,90.242857,0.327429,274.985714,2.776703,0.010075,76.578571,143.457143,6.255286,3.911286,0.338929,103.928571,1.357143,11.942857,0.249714,3.309143,56.143571,20.237357,37.5,999.000000,9.594643,22.321429,84.164286,1.0
2,230221_chamber1_ajc6660_m03bl,1,1,TL,2023-02-21,<NA>,ajc6660,m,bl,3,filler,filler,filler,filler,filler,32.5,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Hypoxia,1020,75.459274,0.318067,237.475957,2.321824,0.009787,90.299313,165.166830,5.461286,4.167291,0.320687,114.897939,0.983317,13.584887,0.353798,1.965093,54.344357,20.244328,37.5,999.658489,9.564779,165.029441,97.894995,1.0
3,230221_chamber1_ajc6660_m03bl,1,1,TL,2023-02-21,<NA>,ajc6660,m,bl,3,filler,filler,filler,filler,filler,32.5,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Ramp Up,56,41.017857,0.276071,147.732143,1.262088,0.008495,93.142857,327.732143,5.308929,4.127857,0.265714,121.053571,0.964286,61.910714,1.626250,1.265179,55.383929,20.241429,37.5,1000.000000,9.584286,7.946429,92.589286,1.0
4,230221_chamber1_ajc6660_m03bl,1,1,TL,2023-02-21,<NA>,ajc6660,m,bl,3,filler,filler,filler,filler,filler,32.5,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Normoxia,350,67.584527,0.302751,224.670487,2.079524,0.009315,84.481375,191.587393,5.609484,3.367106,0.291519,113.567335,1.372493,20.037249,0.504728,2.130344,52.902865,20.269169,37.5,1000.000000,9.547937,52.042980,63.088825,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8,230623_chamber2_ajc7937_f31bl,2,1,DS,2023-06-23,<NA>,ajc7937,f,bl,31,Olfr78F,+/+,filler,filler,filler,21.7,<NA>,<NA>,<NA>,<NA>,No,<NA>,0,"Baseline, Hypoxia, Normoxia, Hypoxia, Normoxia...",Normoxia 2,915,34.969399,0.170874,204.224044,1.611493,0.007874,89.170492,224.911475,3.368044,2.223967,0.170831,127.032787,1.574863,25.626230,0.661148,1.400186,27.526448,20.158973,37.5,1009.000000,9.085027,85.309290,90.873224,1.0
9,230623_chamber2_ajc7937_f31bl,2,1,DS,2023-06-23,<NA>,ajc7937,f,bl,31,Olfr78F,+/+,filler,filler,filler,21.7,<NA>,<NA>,<NA>,<NA>,No,<NA>,0,"Baseline, Hypoxia, Normoxia, Hypoxia, Normoxia...",Ramp 3,249,58.096386,0.206667,281.385542,2.677253,0.009524,88.261044,130.718876,3.550843,3.219197,0.208394,66.594378,1.429719,20.939759,0.866827,2.314900,27.166667,20.160000,37.5,1009.000000,9.083333,25.036145,97.510040,1.0
10,230623_chamber2_ajc7937_f31bl,2,1,DS,2023-06-23,<NA>,ajc7937,f,bl,31,Olfr78F,+/+,filler,filler,filler,21.7,<NA>,<NA>,<NA>,<NA>,No,<NA>,0,"Baseline, Hypoxia, Normoxia, Hypoxia, Normoxia...",Hypercapnia,1374,75.180029,0.250940,299.561953,3.464517,0.011564,86.153790,114.741254,4.240109,4.165539,0.251800,62.762391,1.392857,15.008746,0.838448,2.858404,26.438848,20.184927,37.5,1009.000000,9.073280,170.078717,96.721574,1.0
11,230623_chamber2_ajc7937_f31bl,2,1,DS,2023-06-23,<NA>,ajc7937,f,bl,31,Olfr78F,+/+,filler,filler,filler,21.7,<NA>,<NA>,<NA>,<NA>,No,<NA>,0,"Baseline, Hypoxia, Normoxia, Hypoxia, Normoxia...",Ramp 4,220,64.191781,0.219087,291.223744,2.958147,0.010096,83.621005,123.703196,4.302283,3.445114,0.221233,67.223744,1.397260,

In [35]:
# Get the averages of all protocols for each condition and test_number
# mean_of_conditions_test_number = master_data.groupby(["Conditions", "Test_Number"]).agg(np.mean).reset_index()
# master_data = master_data.append(mean_of_conditions_test_number)

In [36]:
#Get the averages of all protocols for each condition and test_number

In [37]:
# Function to highlight every Test_Number == 1
def highlight_every_number_1(s):
    df = s.copy()
    #set by condition
    mask = (df['Test_Number'] == "1") & (df['Test_Number'].notna())
    df.loc[mask, :] = 'background-color: LightGreen'
    df.loc[~mask,:] = 'color: SlateGrey'
    return df   

In [38]:
condition_list = list(set(master_data["Conditions"].values))
condition_list = [str(condition) for condition in condition_list] # In case of non-string values
print(condition_list)
condition_list.sort()

['Baseline', 'Ramp', 'Ramp 4', 'Normoxia 3', 'Ramp Up', 'Hypoxia 2', 'Hypoxia', 'Hypercapnia', 'Ramp Down', 'Ramp 3', 'Normoxia', 'Normoxia 2', 'Ramp 2']


In [39]:
# Separate master_data based on conditions and write all to one excel sheet
condition_list = list(set(master_data["Conditions"].values))
condition_list = [str(condition) for condition in condition_list] # In case of non-string values
print(condition_list)
condition_list.sort()
writer = pd.ExcelWriter('marcus_breathing_calcs_output.xlsx', engine='xlsxwriter')
for condition in condition_list:
    if condition == '' or pd.isnull(condition):
        continue
    condition_dataframe = master_data[master_data["Conditions"] == condition]
    condition_dataframe = condition_dataframe.reset_index(drop=True)
    condition_dataframe.style.apply(highlight_every_number_1, axis=None).to_excel(writer, sheet_name=str(condition))
writer.close()
print("All Done")

['Baseline', 'Ramp', 'Ramp 4', 'Normoxia 3', 'Ramp Up', 'Hypoxia 2', 'Hypoxia', 'Hypercapnia', 'Ramp Down', 'Ramp 3', 'Normoxia', 'Normoxia 2', 'Ramp 2']
All Done
